# 13 – Health Inequalities: Measuring the Gradient

**Learning Objectives:**
- Understand the Index of Multiple Deprivation (IMD) and how it is constructed
- Distinguish between life expectancy and healthy life expectancy
- Understand the distinction between absolute and relative measures of inequality
- Calculate the Slope Index of Inequality (SII) and Relative Index of Inequality (RII)
- Interpret concentration curves and concentration indices
- Apply these measures to dietary intake and nutrition-related health outcomes

---

## 1. Introduction: Why Measure Inequalities?

Average population health can improve while inequalities widen. Consider:

- Life expectancy in England increased for all groups between 2001-2019
- But the gap between the most and least deprived areas *also* increased

If we only track averages, we miss this divergence. Health inequality metrics help us:

1. **Describe** the current distribution of health across social groups
2. **Monitor** whether policies are reducing or widening gaps
3. **Target** interventions toward those in greatest need

### The Marmot Curve

Health follows a **gradient** — it's not simply that the poorest are unhealthy. Each step up the socioeconomic ladder is associated with better health outcomes.

## 2. Setup

In [ ]:
# ============================================================
# Bootstrap cell (works both locally and in Colab)
# ============================================================

import os
import sys
import pathlib
import subprocess

REPO_URL = "https://github.com/ggkuhnle/phn-epi.git"
REPO_DIR = "phn-epi"

cwd = pathlib.Path.cwd()

if (cwd / "scripts" / "epi_utils.py").is_file():
    repo_root = cwd
elif (cwd.parent / "scripts" / "epi_utils.py").is_file():
    repo_root = cwd.parent
else:
    repo_root = cwd / REPO_DIR
    if not repo_root.is_dir():
        print(f"Cloning repository from {REPO_URL} ...")
        subprocess.run(["git", "clone", REPO_URL, str(repo_root)], check=True)
    else:
        print(f"Using existing repository at {repo_root}")
    os.chdir(repo_root)
    repo_root = pathlib.Path.cwd()

scripts_dir = repo_root / "scripts"
if str(scripts_dir) not in sys.path:
    sys.path.insert(0, str(scripts_dir))

print(f"Repository root: {repo_root}")
print("Bootstrap completed successfully.")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from ipywidgets import FloatSlider, VBox, Output
import ipywidgets as widgets
from IPython.display import display, HTML

from epi_utils import (
    calculate_sii, calculate_rii, calculate_concentration_index,
    plot_concentration_curve, INEQUALITY_EXAMPLE_DATA
)

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [10, 6]
np.random.seed(42)

print("Libraries loaded successfully.")

## 3. The Index of Multiple Deprivation (IMD)

### What is the IMD?

The **Index of Multiple Deprivation (IMD)** is the official measure of relative deprivation for small areas in England. It ranks every **Lower Layer Super Output Area (LSOA)** — approximately 32,844 areas, each containing around 1,500 people.

### The Seven Domains of Deprivation

| Domain | Weight | What it measures |
|--------|--------|------------------|
| **Income** | 22.5% | Proportion of population on low income |
| **Employment** | 22.5% | Proportion involuntarily excluded from work |
| **Education** | 13.5% | Lack of attainment and skills |
| **Health & Disability** | 13.5% | Premature death and impairment |
| **Crime** | 9.3% | Risk of victimisation |
| **Barriers to Housing** | 9.3% | Physical and financial accessibility |
| **Living Environment** | 9.3% | Indoor and outdoor environment quality |

### Important Notes

1. **Relative, not absolute**: IMD ranks areas relative to each other
2. **Area-level, not individual**: Not everyone in a deprived area is deprived (ecological fallacy)
3. **England only**: Scotland, Wales, and Northern Ireland have separate indices

In [ ]:
# IMD domain weights visualisation
imd_domains = pd.DataFrame({
    'Domain': ['Income', 'Employment', 'Education', 'Health & Disability', 
               'Crime', 'Barriers to Housing', 'Living Environment'],
    'Weight (%)': [22.5, 22.5, 13.5, 13.5, 9.3, 9.3, 9.3]
})

fig, ax = plt.subplots(figsize=(10, 6))
colors = plt.cm.Set3(np.linspace(0, 1, 7))
ax.pie(imd_domains['Weight (%)'], labels=imd_domains['Domain'],
       autopct='%1.1f%%', colors=colors, startangle=90)
ax.set_title('Index of Multiple Deprivation: Domain Weights', fontsize=14)
plt.tight_layout()
plt.show()

print("Note: Income and Employment together account for 45% of the overall IMD.")

### Exploring IMD Maps

**Key Resources:**

- [**ONS Interactive IMD Map**](https://dclgapps.communities.gov.uk/imd/iod_index.html) — Explore deprivation by LSOA
- [**CDRC IMD Geodata Pack**](https://data.cdrc.ac.uk/dataset/index-multiple-deprivation-imd) — Download IMD data
- [**PHE Fingertips**](https://fingertips.phe.org.uk/) — Health outcomes by deprivation

In [ ]:
# Display link to interactive IMD map
display(HTML('''
<div style="text-align: center; padding: 20px; background-color: #f5f5f5; border-radius: 10px;">
    <h3>Index of Multiple Deprivation 2019 - England</h3>
    <p><a href="https://dclgapps.communities.gov.uk/imd/iod_index.html" target="_blank">
        Click here to explore the interactive IMD map →
    </a></p>
    <p style="font-size: 12px; color: #666;">Source: MHCLG (2019)</p>
</div>
'''))

In [ ]:
# Most and least deprived local authorities
la_deprivation = pd.DataFrame({
    'Local Authority': ['Blackpool', 'Knowsley', 'Hull', 'Middlesbrough', 'Liverpool',
                        'Hart', 'Wokingham', 'Surrey Heath', 'S. Cambridgeshire', 'Elmbridge'],
    'Rank (of 317)': [1, 2, 3, 4, 5, 317, 316, 315, 314, 313],
    'Category': ['Most deprived'] * 5 + ['Least deprived'] * 5
})

print("Most and Least Deprived Local Authorities (IMD 2019)")
print("=" * 60)
display(la_deprivation)

## 4. Life Expectancy vs Healthy Life Expectancy

### Definitions

- **Life expectancy (LE)**: Average years a newborn would live if current mortality rates persist
- **Healthy life expectancy (HLE)**: Average years expected in "good" or "very good" health

### The Double Burden of Deprivation

The gap between LE and HLE represents **years lived in poor health**. In deprived areas, people:
1. Die younger
2. Spend a larger proportion of their shorter lives in poor health

In [ ]:
# Life expectancy and healthy life expectancy by IMD decile
le_hle_data = pd.DataFrame({
    'IMD Decile': ['1 (Most deprived)', '2', '3', '4', '5', '6', '7', '8', '9', '10 (Least deprived)'],
    'Decile': list(range(1, 11)),
    'LE_male': [73.5, 76.0, 77.5, 78.5, 79.3, 80.0, 80.7, 81.3, 82.0, 83.5],
    'HLE_male': [51.5, 55.0, 58.0, 60.5, 62.5, 64.0, 65.5, 67.0, 68.5, 70.5],
    'LE_female': [78.5, 80.3, 81.5, 82.3, 83.0, 83.7, 84.3, 84.9, 85.5, 86.5],
    'HLE_female': [51.0, 55.0, 58.5, 61.0, 63.0, 65.0, 66.5, 68.0, 69.5, 71.5]
})

le_hle_data['Poor_health_male'] = le_hle_data['LE_male'] - le_hle_data['HLE_male']
le_hle_data['Poor_health_female'] = le_hle_data['LE_female'] - le_hle_data['HLE_female']

print("Life Expectancy and Healthy Life Expectancy by IMD Decile")
display(le_hle_data[['IMD Decile', 'LE_male', 'HLE_male', 'Poor_health_male']])

In [ ]:
# Visualise LE vs HLE
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(10)
ax.bar(x, le_hle_data['HLE_male'], label='Healthy Life Expectancy', color='#27ae60')
ax.bar(x, le_hle_data['Poor_health_male'], bottom=le_hle_data['HLE_male'], 
       label='Years in poor health', color='#e74c3c', alpha=0.7)
ax.set_xticks(x)
ax.set_xticklabels(['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9', 'D10'])
ax.set_xlabel('IMD Decile (D1 = most deprived)')
ax.set_ylabel('Years')
ax.set_title('Males: Life Expectancy by IMD Decile')
ax.legend(loc='lower right')
ax.set_ylim(0, 95)

gap_le = le_hle_data['LE_male'].iloc[-1] - le_hle_data['LE_male'].iloc[0]
gap_hle = le_hle_data['HLE_male'].iloc[-1] - le_hle_data['HLE_male'].iloc[0]
ax.annotate(f'LE gap: {gap_le:.1f} years', xy=(8, 88), fontsize=10)
ax.annotate(f'HLE gap: {gap_hle:.1f} years', xy=(8, 84), fontsize=10, color='#27ae60')

plt.tight_layout()
plt.show()

print(f"\nThe HLE gap ({gap_hle:.0f} years) is nearly TWICE the LE gap ({gap_le:.0f} years).")

### State Pension Age and Healthy Life Expectancy

The State Pension Age is 66 (rising to 67 by 2028). In the most deprived areas, men reach their HLE **14.5 years BEFORE** pension age.

In [ ]:
# Pension age context
pension_age = 66

fig, ax = plt.subplots(figsize=(12, 5))
ax.bar(le_hle_data['Decile'], le_hle_data['HLE_male'], color='#27ae60', alpha=0.8)
ax.axhline(y=pension_age, color='red', linestyle='--', linewidth=2, 
           label=f'State Pension Age ({pension_age})')
ax.set_xlabel('IMD Decile (1 = most deprived)')
ax.set_ylabel('Healthy Life Expectancy (years)')
ax.set_title('Male Healthy Life Expectancy vs State Pension Age')
ax.legend()
ax.set_xticks(le_hle_data['Decile'])
plt.tight_layout()
plt.show()

below_pension = le_hle_data[le_hle_data['HLE_male'] < pension_age]
print(f"In IMD deciles 1-{len(below_pension)}, male HLE is below the state pension age.")

## 5. Measuring Inequality: SII and RII

### Slope Index of Inequality (SII)

The SII fits a regression across the entire gradient, using all data points.

In [ ]:
# Obesity prevalence by deprivation
obesity_data = pd.DataFrame({
    'quintile': ['Q1', 'Q2', 'Q3', 'Q4', 'Q5'],
    'population_share': [0.20, 0.20, 0.20, 0.20, 0.20],
    'obesity_prevalence': [0.34, 0.30, 0.27, 0.24, 0.20]
})

def calculate_sii_manual(data, outcome_col, pop_share_col='population_share'):
    data = data.copy()
    data['cum_pop'] = data[pop_share_col].cumsum()
    data['cum_pop_lag'] = data['cum_pop'].shift(1, fill_value=0)
    data['ridit'] = (data['cum_pop'] + data['cum_pop_lag']) / 2
    slope, intercept, r_value, p_value, _ = stats.linregress(data['ridit'], data[outcome_col])
    return {'sii': slope, 'intercept': intercept, 'data_with_ridit': data}

sii_result = calculate_sii_manual(obesity_data, 'obesity_prevalence')
print(f"SII for obesity: {sii_result['sii']:.3f}")
print(f"\nThis means obesity prevalence decreases by {abs(sii_result['sii']):.1%}")
print(f"moving from most to least deprived.")

In [ ]:
# Visualise SII
fig, ax = plt.subplots(figsize=(10, 6))
data = sii_result['data_with_ridit']
ax.scatter(data['ridit'], data['obesity_prevalence'], s=100, color='steelblue', zorder=5)
x_line = np.linspace(0, 1, 100)
y_line = sii_result['intercept'] + sii_result['sii'] * x_line
ax.plot(x_line, y_line, 'r-', linewidth=2, label=f'SII = {sii_result["sii"]:.3f}')
for i, row in data.iterrows():
    ax.annotate(row['quintile'], (row['ridit'], row['obesity_prevalence']),
                textcoords="offset points", xytext=(0, 10), ha='center')
ax.set_xlabel('Ridit Score (0 = most deprived, 1 = least deprived)')
ax.set_ylabel('Obesity Prevalence')
ax.set_title('Slope Index of Inequality for Obesity')
ax.legend()
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))
plt.tight_layout()
plt.show()

## 6. Discussion Questions

1. **IMD limitations**: Why might the IMD be a poor measure of individual deprivation?

2. **LE vs HLE**: Why is the HLE gap larger than the LE gap? What does this imply for pension policy?

3. **Proportionate universalism**: How might you measure whether a dietary intervention follows this principle?

## 7. Exercises

### Exercise 1: Explore IMD in Your Area

1. Go to the [ONS IMD Explorer](https://dclgapps.communities.gov.uk/imd/iod_index.html)
2. Find your home area (or Reading)
3. Note the IMD decile and domain scores

In [ ]:
# Record your findings
my_area = {
    'Area name': '',
    'IMD Decile': None,
    'Most deprived domain': ''
}

---

## Summary

- **IMD** combines seven domains to rank areas by deprivation
- **Healthy life expectancy** reveals larger inequality gaps than total LE
- The "double burden" means deprived populations live shorter lives AND more years in poor health
- **SII** measures absolute inequality; **RII** expresses it relatively

## Key Resources

- [ONS IMD Interactive Map](https://dclgapps.communities.gov.uk/imd/iod_index.html)
- [PHE Fingertips](https://fingertips.phe.org.uk/)
- [ONS Health State Life Expectancy](https://www.ons.gov.uk/peoplepopulationandcommunity/healthandsocialcare/healthandlifeexpectancies)

## References

- MHCLG (2019). English Indices of Deprivation.
- Marmot M (2010). Fair Society, Healthy Lives.
- Marmot M (2020). Health Equity in England: The Marmot Review 10 Years On.